In [ ]:
from pathlib import Path

In [ ]:
# Installation in case running over Colab
try:
    import google.colab
    %pip install summerepi2==1.1.1
    %pip install estival==0.1.7
    %pip install pylatex==1.4.1
    %pip install kaleido
    ! git clone https://github.com/monash-emu/aust-covid
    %cd aust-covid
    %pip install -e ./
    PROJECT_PATH = Path().resolve()
except:
    PROJECT_PATH = Path().resolve().parent

DATA_PATH = PROJECT_PATH / "data"

In [ ]:
import pandas as pd
pd.options.plotting.backend = "plotly"
from datetime import datetime
import pylatex as pl
from pylatex.utils import NoEscape
from aust_covid.model import build_aust_model
from aust_covid.calibration import DocumentedCalibration
import yaml
import nevergrad as ng

from estival.model import BayesianCompartmentalModel
from estival.optimization.nevergrad import optimize_model
from estival.priors import UniformPrior
from estival.targets import NegativeBinomialTarget

In [ ]:
# Data inputs
aust_data = pd.read_csv(DATA_PATH / "Aus_covid_data.csv", index_col="date")
aust_data.index = pd.to_datetime(aust_data.index)

# Extract national
national_data = aust_data[aust_data["region"] == "AUS"]
smoothed_national_cases = national_data["cases"].rolling(window=7).mean().dropna()

# Extract non-WA
non_wa_data = aust_data.loc[(aust_data["region"] != "AUS") & (aust_data["region"] != "WA")]
non_wa_data = non_wa_data.groupby(non_wa_data.index).sum()
smoothed_non_wa_cases = non_wa_data["cases"].rolling(window=7).mean().dropna()

In [ ]:
# Set up for manual run with supplementary material document
supplement = pl.Document()
supplement.preamble.append(pl.Package("biblatex", options=["sorting=none"]))
supplement.preamble.append(pl.Command("addbibresource", arguments=["austcovid.bib"]))
supplement.preamble.append(pl.Command("title", "Supplemental Appendix"))
supplement.append(NoEscape(r"\maketitle"))

In [ ]:
start_date = datetime(2021, 9, 1)
end_date = datetime(2022, 10, 1)
parameters = {
    "contact_rate": 0.048,
    "infectious_period": 5.0,
    "latent_period": 2.0,
    "cdr": 0.1,
    "seed_rate": 1.0,
    "seed_duration": 1.0,
    "ba1_seed_time": 660.0,
    "ba2_seed_time": 688.0,
    "ba5_seed_time": 720.0,
    "ba2_escape": 0.45,
    "ba5_escape": 0.38,
    "notifs_shape": 2.0,
    "notifs_mean": 4.0,
    "deaths_shape": 2.0,
    "deaths_mean": 20.0,
    "natural_immunity_period": 50.0,
    "ifr_0": 0.0,
    "ifr_5": 0.0,
    "ifr_10": 0.0,
    "ifr_15": 0.0,
    "ifr_20": 0.0,
    "ifr_25": 0.0,
    "ifr_30": 0.0,
    "ifr_35": 0.0,
    "ifr_40": 0.0,
    "ifr_45": 0.0,
    "ifr_50": 0.0,
    "ifr_55": 0.0,
    "ifr_60": 0.0,
    "ifr_65": 0.0,
    "ifr_70": 0.01,
}

aust_model = build_aust_model(start_date, end_date, supplement, add_documentation=False)
aust_model.run(parameters=parameters)

In [ ]:
# Calibration settings
with open(PROJECT_PATH / "inputs/parameters.yml", "r") as param_file:
    param_info = yaml.safe_load(param_file)
param_descriptions = param_info["descriptions"]
param_units = param_info["units"]
param_evidence = param_info["evidence"]

iterations = 500
burn_in = 100
priors = [
    UniformPrior("contact_rate", (0.02, 0.05)),
    UniformPrior("infectious_period", (4.0, 8.0)),
    UniformPrior("ba2_escape", (0.0, 1.0)),
    UniformPrior("ba5_escape", (0.25, 0.7)),
    UniformPrior("ba1_seed_time", (650.0, 670.0)),
    UniformPrior("ba2_seed_time", (678.0, 698.0)),
    UniformPrior("ba5_seed_time", (710.0, 740.0)),
]
smoothed_non_wa_cases_intindex = smoothed_non_wa_cases.copy()
smoothed_non_wa_cases_intindex.index=(smoothed_non_wa_cases.index - aust_model.ref_date).days
targets = [
    NegativeBinomialTarget("notifications", smoothed_non_wa_cases_intindex, 500.0),
]
aust_model = build_aust_model(start_date, end_date, supplement, True)
bcm = BayesianCompartmentalModel(aust_model, parameters, priors, targets)

In [ ]:
orunner = optimize_model(bcm)
for i in range(10):
    rec = orunner.minimize(100)
    print(rec.loss)

In [ ]:
new_params = rec.value[1]
parameters.update(new_params)
aust_model.run(parameters=parameters)
new_params

In [ ]:
# Look at results of manual run
axis_labels = {"index": "time", "value": "cases"}
comparison_df = pd.concat((smoothed_non_wa_cases, aust_model.get_derived_outputs_df()["notifications"]), axis=1)
comparison_df.plot(labels=axis_labels)

In [ ]:
# Run and document the calibration
documented_calib = DocumentedCalibration(
    priors, 
    targets, 
    iterations, 
    burn_in, 
    build_aust_model,
    parameters, 
    param_descriptions, 
    param_units, 
    param_evidence, 
    start_date,
    end_date,
    supplement,
)
documented_calib.get_analysis()
documented_calib.add_calib_table_to_doc()
documented_calib.table_param_results()
documented_calib.graph_param_progression()
documented_calib.add_param_table_to_doc()
documented_calib.compile_doc()

In [ ]:
# Look at a subset of the results of calibration
sample_outputs = documented_calib.get_sample_outputs(50)
pd.concat((smoothed_non_wa_cases, sample_outputs), axis=1).plot(labels=axis_labels)

In [ ]:
# Finish up the supplement document with bibliography
supplement.append(pl.NewPage())
supplement.append(pl.Command("printbibliography"))
supplement.generate_tex(str(PROJECT_PATH / "supplement/supplement"))